In [1]:
import polars as pl
from pathlib import Path
from datasets import load_dataset

d:\anaconda3\envs\power_of_noise\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
golden_dataset = load_dataset('florin-hf/nq_open_gold')
train_golden_dataset = golden_dataset['train']

In [3]:
df = pl.read_parquet("data/corpus.parquet")

In [52]:
random_dataset = []
gold_document_position = 7 # @param {type:"slider", min:0, max:7, step:1}
num_documents = 8
num_random_documents = num_documents - 1

In [53]:
for idx in range(len(train_golden_dataset)):
    question = {}
    question['Question'] = train_golden_dataset[idx]["question"]
    idx_gold_in_corpus = train_golden_dataset[idx]["idx_gold_in_corpus"]
    question['Answers'] = train_golden_dataset[idx]["answers"]
    question['Golden_idx'] = gold_document_position
    corpus_element = df.row(idx_gold_in_corpus)
    sampled_elements = df.sample(n=num_random_documents)
    question['Documents'] = [None] * (num_documents)
    
    # Insert Gold Document
    document = {}
    document['Title'] = corpus_element[0]
    document['Text'] = corpus_element[1]
    question['Documents'][gold_document_position] = document
    # Insert other random Documents
    df_idx = 0
    for i in range(len(question['Documents'])):
        document = {}
        if i == gold_document_position:
            continue
        document["Title"] = sampled_elements[df_idx]["Title"][0]
        document["Text"] = sampled_elements[df_idx]["Text"][0]
        df_idx += 1
        question['Documents'][df_idx] = document

    random_dataset.append(question)
    # print("Terminato idx:", idx)

print(len(random_dataset))

72209


In [54]:
random_dataset_df = pl.DataFrame(random_dataset)
path = f"data/random_dataset_gold_at_{gold_document_position}.parquet"
random_dataset_df.write_parquet(path)